<a href="https://colab.research.google.com/github/yeneralacyr/zerotoheroNLP/blob/main/zerotoheroNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch; print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("guncellenmis_datadeneme.csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))  # num_labels, sınıf sayısına göre ayarlanmalıdır.

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

from transformers import Trainer, TrainingArguments

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=None  # Burada özel bir metrik fonksiyonu belirtebilirsiniz
)

trainer.train()

def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
    outputs = self.model(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
    )
    logits = outputs.logits
    # Calculate the loss
    loss = self.loss_function(logits, labels)
    return {"loss": loss, "logits": logits}

2.3.0+cu121


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1464 [00:00<?, ? examples/s]

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.844277
2,No log,2.554673
3,2.768400,2.453542


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("guncellenmis_datadeneme.csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics  # Add the compute_metrics function here
)

trainer.train()

# Function to classify new sentences
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[list(label_to_id.values()).index(predicted_label_id)]
    return predicted_label

# Test the function with an example sentence
example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")

2.3.0+cu121


FileNotFoundError: [Errno 2] No such file or directory: 'guncellenmis_datadeneme.csv'

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Mevcut PyTorch sürümünü kontrol edin
print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("zerooo_modified.csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükleyin
model_name = "dbmdz/convbert-base-turkish-mc4-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Değerlendirme metriklerini hesaplayan fonksiyon
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=34,
    per_device_eval_batch_size=34,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer oluşturun
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics
)

# Modeli eğitin
trainer.train()

# Modeli ve tokenizer'ı GPU'ya taşıyın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Yeni cümleleri sınıflandıran fonksiyon
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[list(label_to_id.values()).index(predicted_label_id)]
    return predicted_label

# Örnek bir cümle ile fonksiyonu test edin
example_sentence = "Istanbulun ortasında cekmıyor. Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")

2.3.0+cu121


Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4346 [00:00<?, ? examples/s]

Map:   0%|          | 0/1087 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 94.00 MiB. GPU 

In [ ]:
 sentence = "mersin bayisinde çalışanlar ilgisiz"
 classify_sentence(sentence)


In [ ]:
pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
classify_sentence('paketler aşırı pahalı')

In [ ]:
pip install transformers datasets

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Mevcut PyTorch sürümünü kontrol edin
print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("set_modified (1).csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükleyin
model_name = "dbmdz/bert-base-turkish-128k-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize etme fonksiyonu
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Değerlendirme metriklerini hesaplayan fonksiyon
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer oluşturun
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics
)

# Modeli eğitin
trainer.train()

# Modeli ve tokenizer'ı GPU'ya taşıyın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Yeni cümleleri sınıflandıran fonksiyon
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[list(label_to_id.values()).index(predicted_label_id)]
    return predicted_label

# Örnek bir cümle ile fonksiyonu test edin
example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")


2.3.0+cu121


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4346 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# Mevcut PyTorch sürümünü kontrol edin
print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("updated_dataset01.csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükleyin
model_name = "dbmdz/bert-base-turkish-128k-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize etme fonksiyonu
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Değerlendirme metriklerini hesaplayan fonksiyon
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# Erken durdurma için EarlyStoppingCallback'i ayarlayın
# 6. epoch'da overfitting oluyorsa, 5. epoch'ta durdurmak için patience=2 olarak ayarlayın
early_stopping_callback = EarlyStoppingCallback(patience=2, early_stopping_threshold=0.001)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Toplam epoch sayısı 10
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    early_stopping_patience=2,  # Erken durdurma için sabır değerini ayarlayın
)

# Trainer oluşturun
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # Erken durdurma callback'ini ekleyin
)

# Modeli eğitin
trainer.train()

# Modeli ve tokenizer'ı GPU'ya taşıyın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Yeni cümleleri sınıflandıran fonksiyon
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[list(label_to_id.values()).index(predicted_label_id)]
    return predicted_label

# Örnek bir cümle ile fonksiyonu test edin
example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")

2.3.0+cu121


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2129 [00:00<?, ? examples/s]

Map:   0%|          | 0/533 [00:00<?, ? examples/s]

TypeError: EarlyStoppingCallback.__init__() got an unexpected keyword argument 'patience'

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers.trainer_callback import EarlyStoppingCallback

# Mevcut PyTorch sürümünü kontrol edin
print(torch.__version__)

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("updated_dataset01.csv")

# Etiketlerinizi sayısallaştırmak için bir harita oluşturun
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)

# Eğitim ve değerlendirme için DatasetDict oluşturun
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükleyin
model_name = "dbmdz/bert-base-turkish-128k-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize etme fonksiyonu
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

# Tokenize edilmiş veriyi hazırlayın
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Değerlendirme metriklerini hesaplayan fonksiyon
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# Erken durdurma için EarlyStoppingCallback'i ayarlayın
# 6. epoch'da overfitting oluyorsa, 5. epoch'ta durdurmak için patience=2 olarak ayarlayın
early_stopping_callback = EarlyStoppingCallback(patience=2)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Toplam epoch sayısı 10
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer oluşturun
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # Erken durdurma callback'ini ekleyin
)

# Modeli eğitin
trainer.train()

# Modeli ve tokenizer'ı GPU'ya taşıyın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Yeni cümleleri sınıflandıran fonksiyon
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[list(label_to_id.values()).index(predicted_label_id)]
    return predicted_label

# Örnek bir cümle ile fonksiyonu test edin
example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")

2.3.0+cu121


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2129 [00:00<?, ? examples/s]

Map:   0%|          | 0/533 [00:00<?, ? examples/s]

TypeError: EarlyStoppingCallback.__init__() got an unexpected keyword argument 'patience'

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import numpy as np

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                print(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# CSV dosyasını DataFrame olarak yükleyin
data = pd.read_csv("main_data.csv")

# Etiketleri sayısallaştırın (make sure your data has a 'text' and 'label' column)
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(data)

# Eğitim ve doğrulama setlerine ayırın
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükleyin
model_name = "dbmdz/convbert-base-turkish-mc4-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize etme fonksiyonu
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

early_stopping = EarlyStopping(patience=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")


Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3629 [00:00<?, ? examples/s]

Map:   0%|          | 0/908 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.142504,0.599119
2,No log,1.050024,0.646476
3,No log,0.997296,0.657489


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.142504,0.599119
2,No log,1.050024,0.646476
3,No log,0.997296,0.657489
4,1.088700,1.026763,0.667401
5,1.088700,1.051882,0.667401


EarlyStopping counter: 1 out of 3
EarlyStopping counter: 2 out of 3


KeyboardInterrupt: 

In [ ]:
example_sentence = ""
classify_sentence(example_sentence)

'mobil hizmetler'

In [ ]:
pip install transformers[torch] install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [4]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                print(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# Load the CSV file into a DataFrame
data = pd.read_csv("main_dataset.csv")

# Map labels to numeric values (ensure your data has 'text' and 'label' columns)
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(data)

# Split into training and validation sets
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Load model and tokenizer
model_name = "dbmdz/convbert-base-turkish-mc4-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

early_stopping = EarlyStopping(patience=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,  # Reduced learning rate
    per_device_train_batch_size=24,  # Adjusted batch size
    per_device_eval_batch_size=24,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_dir='./logs',  # Ensure logging
    logging_steps=10,  # Log training loss
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")


Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Map:   0%|          | 0/906 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.157200,1.098595,0.640177,0.655626,0.640177,0.585724
2,0.926200,0.987069,0.665563,0.655353,0.665563,0.649714
3,0.912300,0.997062,0.656733,0.652866,0.656733,0.652284
4,0.707600,0.972056,0.674393,0.669951,0.674393,0.671447
5,0.591100,1.012149,0.673289,0.665949,0.673289,0.667918
6,0.510100,1.049554,0.668874,0.664360,0.668874,0.665424


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.157200,1.098595,0.640177,0.655626,0.640177,0.585724
2,0.926200,0.987069,0.665563,0.655353,0.665563,0.649714
3,0.912300,0.997062,0.656733,0.652866,0.656733,0.652284
4,0.707600,0.972056,0.674393,0.669951,0.674393,0.671447
5,0.591100,1.012149,0.673289,0.665949,0.673289,0.667918
6,0.510100,1.049554,0.668874,0.664360,0.668874,0.665424
7,0.545300,1.105265,0.666667,0.658726,0.666667,0.660993


EarlyStopping counter: 3 out of 3
Early stopping activated! No improvement in validation loss for 3 epochs.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predicted label for 'Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın.': mobil hizmetler


In [ ]:
# bu koda model kaydetme ekı eklenecek ve de en ıyı paramtreler gırılecek

from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                print(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# Load the CSV file into a DataFrame
data = pd.read_csv("therandom.csv")

# Map labels to numeric values (ensure your data has 'text' and 'label' columns)
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(data)

# Split into training and validation sets
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Load model and tokenizer
model_name = "dbmdz/convbert-base-turkish-mc4-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

early_stopping = EarlyStopping(patience=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,  # Reduced learning rate
    per_device_train_batch_size=24,  # Adjusted batch size
    per_device_eval_batch_size=24,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_dir='./logs',  # Ensure logging
    logging_steps=10,  # Log training loss
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Map:   0%|          | 0/906 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.211500,1.179247,0.602649,0.666721,0.602649,0.547008
2,0.968700,1.012478,0.647903,0.645208,0.647903,0.644518
3,0.831600,1.022806,0.661148,0.662703,0.661148,0.657028
4,0.658100,1.035588,0.665563,0.658551,0.665563,0.659328
5,0.561800,1.078988,0.649007,0.643886,0.649007,0.643071


EarlyStopping counter: 1 out of 3
EarlyStopping counter: 2 out of 3
EarlyStopping counter: 3 out of 3
Early stopping activated! No improvement in validation loss for 3 epochs.
Predicted label for 'Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın.': mobil hizmetler


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd
import torch
from datasets import load_dataset, Dataset, DatasetDict

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                print(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# CSV dosyasını yükle
data = pd.read_csv("therandom.csv")

# Etiketleri sayısal değerlere dönüştür
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# DataFrame'i Dataset'e dönüştür
dataset = Dataset.from_pandas(data)
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Model ve tokenizer yükle
model_name = "dbmdz/convbert-base-turkish-mc4-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenleme işlevi
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Metrikleri hesapla
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

early_stopping = EarlyStopping(patience=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()

# En iyi modeli yükle onemlı
model_path = training_args.output_dir + "/best_model"
model.save_pretrained(model_path)
print(f"Model saved to {model_path}")

# Modeli kullanarak cümle sınıflandır
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
print(f"Predicted label for '{example_sentence}': {predicted_label}")

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Map:   0%|          | 0/906 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.157300,1.123084,0.631347,0.621913,0.631347,0.584231
2,0.935600,1.002328,0.666667,0.659681,0.666667,0.651178
3,0.905600,0.996720,0.669978,0.662916,0.669978,0.660512
4,0.715400,0.981873,0.681015,0.676365,0.681015,0.677561
5,0.668200,1.016260,0.669978,0.665249,0.669978,0.666270
6,0.532600,1.093124,0.662252,0.656116,0.662252,0.658126
7,0.543400,1.138178,0.663355,0.661441,0.663355,0.661004


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


EarlyStopping counter: 3 out of 3
Early stopping activated! No improvement in validation loss for 3 epochs.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model saved to ./results/best_model


NameError: name 'classify' is not defined

In [3]:
pip install transformers[torch] install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [5]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import optuna
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                logger.info(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# Load the CSV file into a DataFrame
data = pd.read_csv("main_dataset.csv")

# Map labels to numeric values (ensure your data has 'text' and 'label' columns)
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(data)

# Split into training and validation sets
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Load model and tokenizer
model_name = "dbmdz/bert-base-turkish-128k-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

def objective(trial):
    # Hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 24, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Log the hyperparameters for the current trial
    logger.info(f"Trial {trial.number}:")
    logger.info(f"  learning_rate: {learning_rate}")
    logger.info(f"  batch_size: {batch_size}")
    logger.info(f"  num_train_epochs: {num_train_epochs}")
    logger.info(f"  weight_decay: {weight_decay}")

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        logging_dir='./logs',
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        compute_metrics=compute_metrics,
        callbacks=[EarlyStopping(patience=3)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()

    # Return the evaluation metric
    return eval_result["eval_f1"]

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

logger.info("Best trial:")
trial = study.best_trial

logger.info(f"  F1: {trial.value}")
logger.info("  Params: ")
for key, value in trial.params.items():
    logger.info(f"    {key}: {value}")

# Use the best hyperparameters to retrain the model
best_params = trial.params

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    num_train_epochs=best_params["num_train_epochs"],
    weight_decay=best_params["weight_decay"],
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStopping(patience=3)]
)

trainer.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
logger.info(f"Predicted label for '{example_sentence}': {predicted_label}")

# Unit tests
import unittest

class TestModelTraining(unittest.TestCase):
    def test_hyperparameter_tuning(self):
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=1)  # Reduce the number of trials for testing
        trial = study.best_trial
        self.assertTrue(trial.value > 0, "The evaluation metric should be positive")
        self.assertTrue("learning_rate" in trial.params)
        self.assertTrue("batch_size" in trial.params)
        self.assertTrue("num_train_epochs" in trial.params)
        self.assertTrue("weight_decay" in trial.params)

    def test_model_training(self):
        # Use a subset of the dataset for testing to save time
        small_train_dataset = tokenized_datasets['train'].select(range(10))
        small_test_dataset = tokenized_datasets['test'].select(range(10))

        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            learning_rate=1e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=1,
            weight_decay=0.01,
            save_strategy="epoch",
            load_best_model_at_end=True,
            push_to_hub=False,
            logging_dir='./logs',
            logging_steps=10,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=small_train_dataset,
            eval_dataset=small_test_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStopping(patience=1)]
        )

        trainer.train()
        eval_result = trainer.evaluate()
        self.assertTrue(eval_result['eval_accuracy'] > 0, "The evaluation accuracy should be positive")

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Map:   0%|          | 0/906 [00:00<?, ? examples/s]

[I 2024-05-31 15:11:18,746] A new study created in memory with name: no-name-cd967a3e-cd1d-4d54-a015-b355f6d33ed6
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.246500,1.156245,0.571744,0.534432,0.571744,0.519224
2,1.055300,0.986009,0.641280,0.640091,0.641280,0.624913
3,0.950300,0.959601,0.667770,0.668011,0.667770,0.659439
4,0.828600,0.942622,0.674393,0.673986,0.674393,0.662752


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

KeyboardInterrupt: 

In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.0 MB/s eta 0:00:00


In [6]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EarlyStopping(TrainerCallback):
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def on_evaluate(self, args, state, control, **kwargs):
        val_loss = kwargs['metrics']['eval_loss']
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
                logger.info(f"Early stopping activated! No improvement in validation loss for {self.patience} epochs.")
        else:
            self.best_score = score
            self.counter = 0

# Load the CSV file into a DataFrame
data = pd.read_csv("main_dataset.csv")

# Map labels to numeric values (ensure your data has 'text' and 'label' columns)
label_to_id = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_to_id)

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(data)

# Split into training and validation sets
train_test_dataset = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': train_test_dataset['test']
})

# Load model and tokenizer
model_name = "dbmdz/bert-base-turkish-128k-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Tokenize function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Set fixed hyperparameters for training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStopping(patience=3)]
)

# Train the model
trainer.train()

# Save the trained model
model_save_path = "/content/trained_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = "Istanbulun ortasında cekmıyor Allah sızı bıldıgı gıbı yapsın."
predicted_label = classify_sentence(example_sentence)
logger.info(f"Predicted label for '{example_sentence}': {predicted_label}")




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Map:   0%|          | 0/906 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.110200,1.002892,0.644592,0.639647,0.644592,0.612603
2,0.954500,0.940281,0.676600,0.670184,0.676600,0.666861
3,0.815400,0.936388,0.682119,0.676400,0.682119,0.674527


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/transformers/training_ar

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.111840,0.600000,0.920000,0.600000,0.650000
2,No log,1.260975,0.600000,0.920000,0.600000,0.650000


E
ERROR: test_model_training (__main__.TestModelTraining)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-56e58e4adb54>", line 155, in test_model_training
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1885, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2373, in _inner_training_loop
    self.control = self.callback_handler.on_train_end(args, self.state, self.control)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py", line 457, in on_train_end
    return self.call_event("on_train_end", args, state, control)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py", line 498, in call_event
    result = getattr(callback, event)(
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/notebook.py", line 373, in on_trai

In [18]:

model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True).to(device) # Move the input tensor to GPU
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_to_id.keys())[predicted_label_id]
    return predicted_label

example_sentence = ""
predicted_label = classify_sentence(example_sentence)
print(predicted_label)

fatura ve odeme problemleri


In [24]:
!git config --global user.email "alacayiryener@gmail.com "
!git config --global user.name "yeneralacyr"

In [26]:
!git clone https://github.com/yeneralacyr/zerotoheroNLP.git

Cloning into 'zerotoheroNLP'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), done.


In [34]:
!mv trained_model zerotoheroNLP/

In [35]:
%cd /content/zerotoheroNLP/
!git add .
!git commit -m "Add TensorFlow model file"
!git push origin master

/content/zerotoheroNLP
[main 350b30c] Add TensorFlow model file
 5 files changed, 256217 insertions(+)
 create mode 100644 trained_model/config.json
 create mode 100644 trained_model/model.safetensors
 create mode 100644 trained_model/special_tokens_map.json
 create mode 100644 trained_model/tokenizer.json
 create mode 100644 trained_model/vocab.txt
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/yeneralacyr/zerotoheroNLP.git'


In [22]:
from google.colab import files

# Specify the file path
file_path = '/content/trained_model/model.safetensors'

# Download the file
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [21]:
pip freeze > requirements.txt